In [7]:
# test_xi_init_real.py
import numpy as np
from collections import Counter

from env import FPLEnv
from data_utils import load_season_fn, load_gw_fn
from xp import load_models

# ---- Adjust these ----
BASE_DIR = "../data/Fantasy-Premier-League"   # path to the vaastav repo clone root
SEASONS  = ["2023-24"]                        # start with one season you have features for
START_GW = 1                                  # GW to start the episode
models = load_models("../models/rand_forest/classifiers")


env = FPLEnv(
    load_season_fn=load_season_fn,  # now returns season_ctx incl. predictor (if you added it)
    load_gw_fn=load_gw_fn,          # forwards predictor to the pool builder
    seasons=["2023-24"],
    base_dir="../data/Fantasy-Premier-League",
    start_gw=2,                     # if you want transfers to start at GW2
    budget=100.0,
    temperature=0.8,
    transfer_hit=-4.0,
    max_free_transfers=5,
    models=models,                  # <-- pass the 4 models here
)
obs, info = env.reset()


In [8]:
env.squad.players

[Player(pid=352, name='Ederson Santana de Moraes', pos='GK', team='Man City', team_id=13, price=5.5, features=(3.345039996, 1.003511999, 0.611458921, 5.0, 1.0), pooling_metric=88878.0, xP=0.6398868801804469),
 Player(pid=597, name='André Onana', pos='GK', team='Man Utd', team_id=14, price=5.0, features=(5.017559994, 2.676031997, 0.718888147, 3.310813683, 1.0), pooling_metric=158228.0, xP=0.6438328018980136),
 Player(pid=316, name="Amari'i Bell", pos='DEF', team='Luton', team_id=12, price=4.0, features=(0.0, 0.0, 0.0, 3.554068413, 1.0), pooling_metric=773157.0, xP=0.3540019535873501),
 Player(pid=398, name='Luke Shaw', pos='DEF', team='Man Utd', team_id=14, price=5.5, features=(2.78753333, 2.397278664, 0.466748821, 3.310813683, 1.0), pooling_metric=97719.0, xP=0.41944016004729007),
 Player(pid=131, name='Pervis Estupiñán', pos='DEF', team='Brighton', team_id=5, price=5.1, features=(3.902546662, 8.920106656, 0.910709341, 2.506614033, 1.0), pooling_metric=361772.0, xP=0.6534819320138483),

In [9]:
import numpy as np
import copy

def two_transfer_test(env):
    # Reset and get current state
    obs, info = env.reset()
    players = env.squad.players
    xi_idx = list(info["xi_indices"])

    # Pick two worst starters by xP (distinct)
    xi_sorted_asc = sorted(xi_idx, key=lambda i: players[i].xP)
    out1 = xi_sorted_asc[0]
    out2 = xi_sorted_asc[1] if len(xi_sorted_asc) > 1 else 15  # sentinel if somehow no 2nd

    # Helper to find best legal incoming for a given out-slot
    def best_in_for_out(temp_squad, out_slot):
        if out_slot >= 15:
            return None
        out_p = temp_squad.players[out_slot]
        # search pool indices of same position, best xP then cheaper price
        pool_pos_idxs = env.index_map[out_p.pos]
        for j in sorted(pool_pos_idxs, key=lambda k: (env.pool[k].xP, -env.pool[k].price), reverse=True):
            incoming = env.pool[j]
            if temp_squad.can_swap(out_slot, incoming):
                return j
        return None

    # Simulate sequentially on a temp copy (same order env applies)
    temp = copy.deepcopy(env.squad)

    in1 = best_in_for_out(temp, out1)
    if in1 is not None:
        temp.apply_swap(out1, env.pool[in1])

    # Ensure the second out is not the same slot as the first
    if out2 == out1:
        # pick next worst distinct starter
        for idx in xi_sorted_asc[2:]:
            if idx != out1:
                out2 = idx
                break
        else:
            out2 = 15  # no valid second out → sentinel

    in2 = best_in_for_out(temp, out2)
    if in2 is not None:
        temp.apply_swap(out2, env.pool[in2])

    # Build action with proper sentinels for skips
    action = np.array([
        out1 if in1 is not None else 15,
        in1  if in1 is not None else len(env.pool),
        out2 if (out2 < 15 and in2 is not None) else 15,
        in2  if (out2 < 15 and in2 is not None) else len(env.pool),
    ], dtype=int)

    print("Proposed transfers:")
    if in1 is not None:
        print(f"  T1: OUT {players[out1].name}  ->  IN {env.pool[in1].name}")
    else:
        print("  T1: skip")
    if (out2 < 15) and (in2 is not None):
        print(f"  T2: OUT {players[out2].name}  ->  IN {env.pool[in2].name}")
    else:
        print("  T2: skip")

    obs, reward, done, truncated, info = env.step(action)
    print("Step info:", info)
    print("Reward:", reward)
    return obs, reward, done, truncated, info

# Usage:
# obs, reward, done, truncated, info = two_transfer_test(env)


In [11]:
outcome = two_transfer_test(env)
print("Info:")
print(outcome[4])  # info contains the step details
print("next pool size", env.pool_size)

Proposed transfers:
  T1: OUT Kieran Trippier  ->  IN Kyle Walker
  T2: OUT Eberechi Eze  ->  IN Luis Díaz
Step info: {'season': '2023-24', 'current_gw': 3, 'team_xP_expected': 62.0, 'captain': 'Bukayo Saka', 'vice_captain': 'Luis Díaz', 'xi_indices': [0, 6, 4, 2, 7, 8, 10, 9, 11, 12, 14], 'bank': 0.9000000000000004, 'free_transfers': 0, 'action': 'step', 'points_hit': -4.0}
Reward: -2.0
Info:
{'season': '2023-24', 'current_gw': 3, 'team_xP_expected': 62.0, 'captain': 'Bukayo Saka', 'vice_captain': 'Luis Díaz', 'xi_indices': [0, 6, 4, 2, 7, 8, 10, 9, 11, 12, 14], 'bank': 0.9000000000000004, 'free_transfers': 0, 'action': 'step', 'points_hit': -4.0}
next pool size 80


In [5]:
print(env.current_gw)
env.squad.players

2


[Player(pid=524, name='Alphonse Areola', pos='GK', team='West Ham', team_id=19, price=4.0, features=(1.672519998, 1.170763999, 0.682488298, 2.493385967, 1.0), pooling_metric=93606.0, xP=0.6371754590700588),
 Player(pid=597, name='André Onana', pos='GK', team='Man Utd', team_id=14, price=5.0, features=(5.017559994, 2.676031997, 0.718888147, 3.310813683, 1.0), pooling_metric=158228.0, xP=0.6438328018980135),
 Player(pid=131, name='Pervis Estupiñán', pos='DEF', team='Brighton', team_id=5, price=5.1, features=(3.902546662, 8.920106656, 0.910709341, 2.506614033, 1.0), pooling_metric=361772.0, xP=0.6534819320138483),
 Player(pid=473, name='George Baldock', pos='DEF', team='Sheffield Utd', team_id=17, price=4.0, features=(1.115013332, 0.557506666, 0.730817263, 2.0, 0.5), pooling_metric=100654.0, xP=0.41930714395092106),
 Player(pid=29, name='Benjamin White', pos='DEF', team='Arsenal', team_id=1, price=5.5, features=(0.0, 0.0, 0.0, 5.0, 1.0), pooling_metric=812755.0, xP=0.6662881035465901),
 P